In [1]:
# libraries
import pandas as pd

In [49]:
#read Json file

df = pd.read_json("rentalTrend.json")

In [50]:
df.head()

,FSA,bathrooms,bedrooms,description,furnished,id,image,lat,long,pet_friendly,post_published_date,postal_code,price,rental_type,source,sqft,title,url
0,M5B,0,1,1BR / 0Ba furnished apartment/ 1br -Brand new ...,True,c_7195819164,None,43.657200,-79.378300,False,2020-09-21,M5B 1Y2,995,apartment,craigslist,,1bedroom+1den near Eaton Center,https://toronto.craigslist.org/tor/apa/d/toron...
1,M5R,0,1,1BR / 0Ba furnished apartment/ 1br -Spectacula...,True,c_7195700072,None,43.673600,-79.403500,False,2020-09-21,M5R 2R8,1220,condo,craigslist,,Top Floor Condo For Rent,https://toronto.craigslist.org/tor/apa/d/centr...
2,M6H,2,2,2BR / 2Ba 1100ft2 available nov 1 cats are OK ...,None,c_7200564066,https://images.craigslist.org/00202_iyRnxvT5sc...,43.665500,-79.437800,True,2020-09-21,M6H 4B9,3000,townhouse,craigslist,1100,Contemporary 2+1 bedroom townhouse w/ parking,https://toronto.craigslist.org/tor/apa/d/west-...
3,M5V,2,2,2BR / 2Ba available nov 15 loft w/d in unit at...,None,c_7196251096,https://images.craigslist.org/00O0O_fAJia8pnVQ...,43.647329,-79.395794,False,2020-09-21,M5V,4500,loft,craigslist,,RICHMOND/SPADINA-2 BED/2 BATH/1 PARKING-CAMDEN ST,https://toronto.craigslist.org/tor/apa/d/toron...
4,M6G,0,1,1BR / 0Ba furnished apartment/ 1br -Newly reno...,True,c_7195818373,None,43.668300,-79.420500,False,2020-09-21,M6G 3B4,977,apartment,craigslist,,1BDR Downtown Toronto apartment unit for rent,https://toronto.craigslist.org/tor/apa/d/downt...


# Averages by FSA

In [51]:
df["first"] = df['FSA'].astype(str).str[0]

In [52]:
df_cleaned_full = df.loc[(df['first'] == "M") 
                         & (df["price"] < 10000) 
                         & (df["bedrooms"] != 0) 
                         & (df["bedrooms"]!= "") 
                         & (df["post_published_date"] > "2020-09-18")]

In [53]:
df_cleaned_oneortwo = df_cleaned_full.loc[(df_cleaned_full["bedrooms"] == 1) | (df_cleaned_full["bedrooms"] == 2)]

In [54]:
#find mean by no. of bedrooms and FSA - one/Two bedrooms
df_aggregate_oneortwo = df_cleaned_oneortwo.groupby(["FSA", "post_published_date", "bedrooms"])["price"].mean()

In [55]:
df_intermediate_oneortwo= df_aggregate_oneortwo.reset_index()

In [56]:
df_overall_oneortwo = df_cleaned_oneortwo.groupby(["post_published_date", "bedrooms"])["price"].mean()

In [57]:
df_overall_intermediate_oneortwo= df_overall_oneortwo.reset_index()

In [58]:
df_overall_intermediate_oneortwo["FSA"] = "Overall"

In [59]:
#find mean by no. of bedrooms and FSA - one/Two bedrooms
df_cleaned_threeormore = df_cleaned_full.loc[df_cleaned_full["bedrooms"] > 2]

In [60]:
df_overall_threeormore = df_cleaned_threeormore.groupby(["post_published_date"])["price"].mean()

In [61]:
df_overall_intermediate_threeormore= df_overall_threeormore.reset_index()

In [62]:
df_overall_intermediate_threeormore["FSA"] = "Overall"

In [63]:
df_overall_intermediate_threeormore["bedrooms"] = "3 or More"

In [64]:
df_threeormore = df_cleaned_threeormore.groupby(["FSA", "post_published_date"])["price"].mean()

In [65]:
df_threeormore_intermediate= df_threeormore.reset_index()

In [66]:
df_threeormore_intermediate["bedrooms"] = "3 or More"

In [67]:
#combine datasets
full_df = pd.concat([df_intermediate_oneortwo, df_threeormore_intermediate, df_overall_intermediate_oneortwo, df_overall_intermediate_threeormore], sort=True, ignore_index=True)

In [68]:
full_df.rename(columns={"price" : "average_price"}, inplace=True)

In [69]:
#save to file as Json
full_df.to_json("RentalPrice.json")

In [70]:
#save to file as CSV
full_df.to_csv("RentalPrice.csv")

# Find averages by drivers/clusters

In [71]:
#read in cluster dataframe
df_cluster = pd.read_csv("RentalTrendWClusterID.csv")

In [72]:
df_cluster.head()

,Unnamed: 0,fsa,bathrooms,bedrooms,description,furnished,id,image,lat,long,...,Food & Housing,Education & Employment,Avg_Age,avg_income,Assault,Auto Theft,Break and Enter,Robbery,Theft Over,cluster
0,0,M5B,0.0,1.0,1BR / 0Ba furnished apartment/ 1br -Brand new ...,YES,c_7195819164,1,43.657200,-79.378300,...,9,4,36.6,46121.436,1212,96,280,246,65,2.0
1,30,M5B,1.0,0.0,0BR / 1Ba 600ft2 available oct 31 furnished co...,YES,c_7199431318,1,43.654351,-79.379160,...,9,4,36.6,46121.436,1212,96,280,246,65,1.0
2,66,M5B,1.0,0.0,0BR / 1Ba 600ft2 available oct 30 furnished co...,YES,c_7200468956,1,43.654382,-79.378946,...,9,4,36.6,46121.436,1212,96,280,246,65,1.0
3,78,M5B,1.5,1.0,1BR / 1.5Ba 564ft2 available now cats are OK -...,NOT_MENTIONED,c_7200513078,1,43.655147,-79.374925,...,9,4,36.6,46121.436,1212,96,280,246,65,0.0
4,142,M5B,1.0,0.0,0BR / 1Ba available now cats are OK - purrr do...,YES,c_7200327071,1,43.657200,-79.378300,...,9,4,36.6,46121.436,1212,96,280,246,65,0.0


In [73]:
df_cluster.columns

Index(['Unnamed: 0', 'fsa', 'bathrooms', 'bedrooms', 'description',
       'furnished', 'id', 'image', 'lat', 'long', 'pet_friendly',
       'post_published_date', 'pred', 'price', 'rental_type', 'sqft', 'title',
       'url', 'Community Services', 'Health Services', 'Financial Services',
       'Law & Government', 'Transportation', 'Food & Housing',
       'Education & Employment', 'Avg_Age', 'avg_income', 'Assault',
       'Auto Theft', 'Break and Enter', 'Robbery', 'Theft Over', 'cluster'],
      dtype='object')

In [74]:
#select columns of interest
df_cluster = df_cluster[["id", "cluster", "image", "pet_friendly","furnished"]]

In [75]:
df_cluster

,id,cluster,image,pet_friendly,furnished
0,c_7195819164,2.0,1,False,YES
1,c_7199431318,1.0,1,False,YES
2,c_7200468956,1.0,1,False,YES
3,c_7200513078,0.0,1,True,NOT_MENTIONED
4,c_7200327071,0.0,1,True,YES
...,...,...,...,...,...
11853,c_7221283051,2.0,1,False,NOT_MENTIONED
11854,c_7222926094,1.0,1,False,NOT_MENTIONED
11855,c_7222920460,2.0,1,False,NOT_MENTIONED
11856,c_7222873722,2.0,1,False,NOT_MENTIONED


In [76]:
#update naming
df_cluster.rename(columns={"image":"image_boolean", "pet_friendly": "pet_friendly_cl", "furnished": "furnished_update"}, inplace=True)

In [77]:
df_cluster

,id,cluster,image_boolean,pet_friendly_cl,furnished_update
0,c_7195819164,2.0,1,False,YES
1,c_7199431318,1.0,1,False,YES
2,c_7200468956,1.0,1,False,YES
3,c_7200513078,0.0,1,True,NOT_MENTIONED
4,c_7200327071,0.0,1,True,YES
...,...,...,...,...,...
11853,c_7221283051,2.0,1,False,NOT_MENTIONED
11854,c_7222926094,1.0,1,False,NOT_MENTIONED
11855,c_7222920460,2.0,1,False,NOT_MENTIONED
11856,c_7222873722,2.0,1,False,NOT_MENTIONED


In [204]:
#read Json file

df = pd.read_json("rentalTrend.json")

In [205]:
df["first"] = df['FSA'].astype(str).str[0]

In [208]:
df = df.loc[(df["price"] < 10000) & (df["price"] >= 200) & (df["first"] == "M")]

In [209]:
df.FSA.unique()

array(['M5B', 'M5R', 'M6H', 'M5V', 'M6G', 'M4C', 'M5N', 'M8Y', 'M1J',
       'M5S', 'M1P', 'M6B', 'M5A', 'M2M', 'M1N', 'M6P', 'M2R', 'M2N',
       'M6J', 'M6K', 'M6E', 'M4Y', 'M6N', 'M7A', 'M1L', 'M5J', 'M6R',
       'M4E', 'M2H', 'M6C', 'M5G', 'M4N', 'M6A', 'M4V', 'M6S', 'M3A',
       'M4T', 'M4K', 'M4L', 'M5E', 'M5T', 'M9W', 'M5H', 'M1E', 'M4W',
       'M8V', 'M4X', 'M4P', 'M1H', 'M1M', 'M4S', 'M5C', 'M6M', 'M4M',
       'M9M', 'M2J', 'M9R', 'M1S', 'M5P', 'M8Z', 'M4J', 'M6L', 'M9V',
       'M3L', 'M3K', 'M3C', 'M2K', 'M3M', 'M8W', 'M5M', 'M9B', 'M9N',
       'M3H', 'M9A', 'M3J', 'M1B', 'M9C', 'M1W', 'M4R', 'M4B', 'M1T',
       'M1K', 'M8X', 'M4G', 'M1G', 'M2L', 'M1V', 'M1R', 'M1C', 'M4H',
       'M3N', 'M1X', 'M3B', 'M2P', 'M4A', 'M9P'], dtype=object)

In [210]:
df.drop(columns="first", inplace=True)

C:\Users\megsm\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [211]:
df.head()

,FSA,bathrooms,bedrooms,description,furnished,id,image,lat,long,pet_friendly,post_published_date,postal_code,price,rental_type,source,sqft,title,url
0,M5B,0,1,1BR / 0Ba furnished apartment/ 1br -Brand new ...,True,c_7195819164,None,43.657200,-79.378300,False,2020-09-21,M5B 1Y2,995,apartment,craigslist,,1bedroom+1den near Eaton Center,https://toronto.craigslist.org/tor/apa/d/toron...
1,M5R,0,1,1BR / 0Ba furnished apartment/ 1br -Spectacula...,True,c_7195700072,None,43.673600,-79.403500,False,2020-09-21,M5R 2R8,1220,condo,craigslist,,Top Floor Condo For Rent,https://toronto.craigslist.org/tor/apa/d/centr...
2,M6H,2,2,2BR / 2Ba 1100ft2 available nov 1 cats are OK ...,None,c_7200564066,https://images.craigslist.org/00202_iyRnxvT5sc...,43.665500,-79.437800,True,2020-09-21,M6H 4B9,3000,townhouse,craigslist,1100,Contemporary 2+1 bedroom townhouse w/ parking,https://toronto.craigslist.org/tor/apa/d/west-...
3,M5V,2,2,2BR / 2Ba available nov 15 loft w/d in unit at...,None,c_7196251096,https://images.craigslist.org/00O0O_fAJia8pnVQ...,43.647329,-79.395794,False,2020-09-21,M5V,4500,loft,craigslist,,RICHMOND/SPADINA-2 BED/2 BATH/1 PARKING-CAMDEN ST,https://toronto.craigslist.org/tor/apa/d/toron...
4,M6G,0,1,1BR / 0Ba furnished apartment/ 1br -Newly reno...,True,c_7195818373,None,43.668300,-79.420500,False,2020-09-21,M6G 3B4,977,apartment,craigslist,,1BDR Downtown Toronto apartment unit for rent,https://toronto.craigslist.org/tor/apa/d/downt...


In [212]:
#merge
combined_df = pd.merge(df_cluster, df, on ="id")

In [213]:
combined_df.head(20)

,id,cluster,image_boolean,pet_friendly_cl,furnished_update,FSA,bathrooms,bedrooms,description,furnished,...,long,pet_friendly,post_published_date,postal_code,price,rental_type,source,sqft,title,url
0,c_7195819164,2.0,1,False,YES,M5B,0,1,1BR / 0Ba furnished apartment/ 1br -Brand new ...,True,...,-79.378300,False,2020-09-21,M5B 1Y2,995,apartment,craigslist,,1bedroom+1den near Eaton Center,https://toronto.craigslist.org/tor/apa/d/toron...
1,c_7199431318,1.0,1,False,YES,M5B,1,0,0BR / 1Ba 600ft2 available oct 31 furnished co...,True,...,-79.379160,False,2020-09-21,M5B 1V8,1699,condo,craigslist,600,"Core Downtown, FULLY FURNISHED Studio Condo w/...",https://toronto.craigslist.org/tor/apa/d/toron...
2,c_7200468956,1.0,1,False,YES,M5B,1,0,0BR / 1Ba 600ft2 available oct 30 furnished co...,True,...,-79.378946,False,2020-09-21,M5B 1V8,1699,condo,craigslist,600,"FULLY FURNISHED STUDIO CONDO w/Internet, W/D, ...",https://toronto.craigslist.org/tor/apa/d/toron...
3,c_7200513078,0.0,1,True,NOT_MENTIONED,M5B,1.5,1,1BR / 1.5Ba 564ft2 available now cats are OK -...,None,...,-79.374925,True,2020-09-21,M5B 1B3,2400,loft,craigslist,564,Brand New Loft Style Condo (Near Yonge & Dundas),https://toronto.craigslist.org/tor/apa/d/toron...
4,c_7200327071,0.0,1,True,YES,M5B,1,0,0BR / 1Ba available now cats are OK - purrr do...,True,...,-79.378300,True,2020-09-21,M5B 1Y2,1680,condo,craigslist,,brand new never lived in before locker include...,https://toronto.craigslist.org/tor/apa/d/toron...
5,c_7188364044,1.0,1,False,YES,M5B,1,1,1BR / 1Ba 575ft2 available now furnished condo...,True,...,-79.378300,False,2020-09-21,M5B 1Y2,2695,condo,craigslist,575,"Modern 1 Bed, 1 Bath with Balcony and Spacious...",https://toronto.craigslist.org/tor/apa/d/toron...
6,c_7185552789,1.0,1,False,YES,M5B,1,1,1BR / 1Ba 650ft2 available oct 1 furnished con...,True,...,-79.378300,False,2020-09-21,M5B 1Y2,2495,condo,craigslist,650,"Stylish 1 Bedroom, 1 Bath with Stainless Steel...",https://toronto.craigslist.org/tor/apa/d/toron...
7,c_7188497307,1.0,1,False,YES,M5B,2,2,2BR / 2Ba 843ft2 available now condo w/d in un...,True,...,-79.380765,False,2020-09-21,M5B 1J2,3500,condo,craigslist,843,Balmuto: Fully Furnished 2 Bedroom Corner Cond...,https://toronto.craigslist.org/tor/apa/d/toron...
8,c_7188395191,1.0,1,False,NOT_MENTIONED,M5B,1,1,1BR / 1Ba 505ft2 available now condo w/d in un...,None,...,-79.378300,False,2020-09-21,M5B 1Y2,1800,condo,craigslist,505,The Met II: Modern 1 Bdrm Unfurnished Condo fo...,https://toronto.craigslist.org/tor/apa/d/toron...
9,c_7198694875,1.0,1,False,YES,M5B,1,0,0BR / 1Ba 500ft2 available now condo w/d in un...,True,...,-79.378300,False,2020-09-21,M5B 1Y2,1800,condo,craigslist,500,PANTAGES condo: Fully Furnished Studio/Bachelo...,https://toronto.craigslist.org/tor/apa/d/toron...


In [196]:
combined_df.to_csv("Full_raw.csv")

## Bathrooms

In [214]:
#find mean by no. of bathrooms and cluster 
df_aggregate_cluster_bathrooms= combined_df.groupby(["cluster", "bathrooms"])["price"].mean()

In [215]:
df_aggregate_cluster_bathrooms

cluster                  bathrooms
0.0                      0.0          1066.480000
                         1.0          1885.659856
                         1.5          2442.590476
                         2.0          2813.220211
                         2.5          3029.464286
                         3.0          3648.075000
                         3.5          5765.000000
                         4.0          4208.000000
                         5.0          4613.500000
                                      1696.030303
1.0                      0.0          1433.333333
                         1.0          2029.535289
                         1.5          2223.290909
                         2.0          2761.748099
                         2.5          5202.142857
                         3.0          4450.937500
                                      2015.000000
2.0                      0.0          1589.552632
                         1.0          1861.333123
               

In [216]:
#counts
count = combined_df.groupby(["cluster", "bathrooms"]).count()

In [217]:
count

id  image_boolean  pet_friendly_cl  \
cluster                 bathrooms                                         
0.0                     0.0         125            125              125   
                        1.0        2496           2496             2496   
                        1.5         105            105              105   
                        2.0         663            663              663   
                        2.5          28             28               28   
                        3.0          40             40               40   
                        3.5           3              3                3   
                        4.0           5              5                5   
                        5.0           2              2                2   
                                     33             33               33   
1.0                     0.0           6              6                6   
                        1.0        2182           2182             2182   
                        1.5         110            110              110   
                        2.0        1052           1052             1052   
                        2.5           7              7                7   
                        3.0          32             32               32   
                                     35             35               35   
2.0                     0.0          38             38               38   
                        1.0        3164           3164             3164   
                        1.5         113            113              113   
                        2.0         503            503              503   
                        2.5          27             27               27   
                        3.0         149            149              149   
                        3.5           5              5                5   
                        4.0          21             21               21   
                        5.0           9              9                9   
                        6.0           1              1                1   
                        8.5           1              1                1   
                                    192            192              192   
Not Feasible to predict 1.0          15             15               15   
                        1.5           1              1                1   
                        2.0           3              3                3   
                        2.5           2              2                2   
                        3.0           3              3                3   
                        4.0           5              5                5   
                        6.0           1              1                1   
                                      1              1                1   

                                   furnished_update   FSA  bedrooms  \
cluster                 bathrooms                                     
0.0                     0.0                     125   125       125   
                        1.0                    2496  2496      2496   
                        1.5                     105   105       105   
                        2.0                     663   663       663   
                        2.5                      28    28        28   
                        3.0                      40    40        40   
                        3.5                       3     3         3   
                        4.0                       5     5         5   
                        5.0                       2     2         2   
                                                 33    33        33   
1.0                     0.0                       6     6         6   
                        1.0                    2182  2182      2182   
                        1.5                     110   110       110   
                        2.0                   

In [218]:
#counts
combined_df.groupby("bathrooms").count()

,id,cluster,image_boolean,pet_friendly_cl,furnished_update,FSA,bedrooms,description,furnished,image,...,long,pet_friendly,post_published_date,postal_code,price,rental_type,source,sqft,title,url
bathrooms,,,,,,,,,,,,,,,,,,,,,
0.0,169,169,169,169,169,169,169,169,156,79,...,169,169,169,169,169,169,169,169,169,169
1.0,7857,7857,7857,7857,7857,7857,7857,7857,6060,7741,...,7857,7857,7857,7857,7857,7857,7857,7857,7857,7857
1.5,329,329,329,329,329,329,329,329,273,322,...,329,329,329,329,329,329,329,329,329,329
2.0,2221,2221,2221,2221,2221,2221,2221,2221,1642,2198,...,2221,2221,2221,2221,2221,2221,2221,2221,2221,2221
2.5,64,64,64,64,64,64,64,64,53,64,...,64,64,64,64,64,64,64,64,64,64
3.0,224,224,224,224,224,224,224,224,165,222,...,224,224,224,224,224,224,224,224,224,224
3.5,8,8,8,8,8,8,8,8,7,8,...,8,8,8,8,8,8,8,8,8,8
4.0,31,31,31,31,31,31,31,31,25,31,...,31,31,31,31,31,31,31,31,31,31
5.0,11,11,11,11,11,11,11,11,8,11,...,11,11,11,11,11,11,11,11,11,11


In [219]:
df_aggregate_cluster_bathrooms_int = df_aggregate_cluster_bathrooms.reset_index()

In [220]:
df_aggregate_cluster_bathrooms_int

,cluster,bathrooms,price
0,0.0,0,1066.480000
1,0.0,1,1885.659856
2,0.0,1.5,2442.590476
3,0.0,2,2813.220211
4,0.0,2.5,3029.464286
5,0.0,3,3648.075000
6,0.0,3.5,5765.000000
7,0.0,4,4208.000000
8,0.0,5,4613.500000
9,0.0,,1696.030303


In [221]:
#find mean by no. of bathrooms /overall
df_aggregate_overall_bathrooms= combined_df.groupby("bathrooms")["price"].mean()

In [222]:
test = df.groupby("bathrooms")["price"].mean()

In [223]:
test

bathrooms
0.0    1197.118343
1.0    1914.838861
1.5    2296.036474
2.0    2843.423835
2.5    3619.687500
3.0    3480.495536
3.5    4904.375000
4.0    4931.096774
5.0    6293.363636
6.0    4825.000000
8.5    6500.000000
       1729.310345
Name: price, dtype: float64

In [224]:
df_aggregate_overall_bathrooms

bathrooms
0.0    1197.118343
1.0    1914.633957
1.5    2296.036474
2.0    2843.902747
2.5    3619.687500
3.0    3480.495536
3.5    4904.375000
4.0    4931.096774
5.0    6293.363636
6.0    4825.000000
8.5    6500.000000
       1729.310345
Name: price, dtype: float64

In [225]:
df_aggregate_overall_bathrooms_int= df_aggregate_overall_bathrooms.reset_index()

In [226]:
df_aggregate_overall_bathrooms_int["cluster"] ="overall"

In [227]:
df_aggregate_overall_bathrooms_int = df_aggregate_overall_bathrooms_int[["cluster", "bathrooms", "price"]]

In [228]:
df_aggregate_overall_bathrooms_int

,cluster,bathrooms,price
0,overall,0,1197.118343
1,overall,1,1914.633957
2,overall,1.5,2296.036474
3,overall,2,2843.902747
4,overall,2.5,3619.687500
5,overall,3,3480.495536
6,overall,3.5,4904.375000
7,overall,4,4931.096774
8,overall,5,6293.363636
9,overall,6,4825.000000


In [229]:
#combine datasets
full_bathrooms = pd.concat([df_aggregate_overall_bathrooms_int, df_aggregate_cluster_bathrooms_int])

In [230]:
full_bathrooms.dropna(inplace=True)

full_bathrooms.reset_index(inplace=True)

In [231]:
full_bathrooms.drop(columns="index", inplace=True)

In [232]:
full_bathrooms["Measure"] ="No_Bathrooms"

In [233]:
full_bathrooms.rename(columns ={"bathrooms":"measure_value"}, inplace=True)

In [234]:
full_bathrooms.to_csv("bathroomAgg.csv")

## Bedrooms

In [235]:
#find mean by no. of bedrooms and cluster 
df_aggregate_cluster_bedrooms= combined_df.groupby(["cluster", "bedrooms"])["price"].mean()

In [267]:
combined_df.groupby(["cluster", "bedrooms"]).count()

id  image_boolean  pet_friendly_cl  \
cluster                 bedrooms                                         
0.0                     0.0        194            194              194   
                        1.0       1854           1854             1854   
                        2.0       1045           1045             1045   
                        3.0        304            304              304   
                        4.0         61             61               61   
                        5.0         10             10               10   
                        6.0          1              1                1   
                                    31             31               31   
1.0                     0.0        249            249              249   
                        1.0       1886           1886             1886   
                        2.0       1160           1160             1160   
                        3.0         92             92               92   
                        4.0          1              1                1   
                                    36             36               36   
2.0                     0.0        350            350              350   
                        1.0       1654           1654             1654   
                        2.0       1271           1271             1271   
                        3.0        574            574              574   
                        4.0        118            118              118   
                        5.0         53             53               53   
                        6.0          2              2                2   
                        7.0          1              1                1   
                        8.0          1              1                1   
                                   199            199              199   
Not Feasible to predict 0.0          5              5                5   
                        1.0          8              8                8   
                        2.0          4              4                4   
                        3.0          3              3                3   
                        4.0          6              6                6   
                        5.0          2              2                2   
                        6.0          3              3                3   

                                  furnished_update   FSA  bathrooms  \
cluster                 bedrooms                                      
0.0                     0.0                    194   194        194   
                        1.0                   1854  1854       1854   
                        2.0                   1045  1045       1045   
                        3.0                    304   304        304   
                        4.0                     61    61         61   
                        5.0                     10    10         10   
                        6.0                      1     1          1   
                                                31    31         31   
1.0                     0.0                    249   249        249   
                        1.0                   1886  1886       1886   
                        2.0                   1160  1160       1160   
                        3.0                     92    92         92   
                        4.0                      1     1          1   
                                                36    36         36   
2.0                     0.0                    350   350        350   
                        1.0                   1654  1654       1654   
                        2.0                   1271  1271       1271   
                        3.0                    574   574        574   
                        4.0                    118   118        118   
                        5.0                     53    53         53   
                        6.0    

In [236]:
df_aggregate_cluster_bedrooms_int = df_aggregate_cluster_bedrooms.reset_index()

In [237]:
#find mean by no. of bedrooms /overall
df_aggregate_overall_bedrooms= combined_df.groupby("bedrooms")["price"].mean()

In [266]:
combined_df.groupby("bedrooms").count()

,id,cluster,image_boolean,pet_friendly_cl,furnished_update,FSA,bathrooms,description,furnished,image,...,long,pet_friendly,post_published_date,postal_code,price,rental_type,source,sqft,title,url
bedrooms,,,,,,,,,,,,,,,,,,,,,
0.0,798,798,798,798,798,798,798,798,614,790,...,798,798,798,798,798,798,798,798,798,798
1.0,5402,5402,5402,5402,5402,5402,5402,5402,4230,5247,...,5402,5402,5402,5402,5402,5402,5402,5402,5402,5402
2.0,3480,3480,3480,3480,3480,3480,3480,3480,2684,3428,...,3480,3480,3480,3480,3480,3480,3480,3480,3480,3480
3.0,973,973,973,973,973,973,973,973,683,950,...,973,973,973,973,973,973,973,973,973,973
4.0,186,186,186,186,186,186,186,186,131,183,...,186,186,186,186,186,186,186,186,186,186
5.0,65,65,65,65,65,65,65,65,44,64,...,65,65,65,65,65,65,65,65,65,65
6.0,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
7.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [238]:
df_aggregate_overall_bedrooms_int= df_aggregate_overall_bedrooms.reset_index()

In [239]:
df_aggregate_overall_bedrooms_int["cluster"] ="overall"

In [240]:
df_aggregate_overall_bedrooms_int = df_aggregate_overall_bedrooms_int[["cluster", "bedrooms", "price"]]

In [241]:
#combine datasets
full_bedrooms = pd.concat([df_aggregate_overall_bedrooms_int, df_aggregate_cluster_bedrooms_int])

In [242]:
full_bedrooms.dropna(inplace=True)

full_bedrooms.reset_index(inplace=True)

In [243]:
full_bedrooms.drop(columns="index", inplace=True)

In [244]:
full_bedrooms["Measure"] ="No_Bedrooms"

In [245]:
full_bedrooms.rename(columns ={"bedrooms":"measure_value"}, inplace=True)

## Type

In [246]:
#find mean by rental type and cluster 
df_aggregate_cluster_type= combined_df.groupby(["cluster", "rental_type"])["price"].mean()

df_aggregate_cluster_type_int = df_aggregate_cluster_type.reset_index()

#find mean by rental type /overall
df_aggregate_overall_type= combined_df.groupby("rental_type")["price"].mean()

df_aggregate_overall_type_int= df_aggregate_overall_type.reset_index()

df_aggregate_overall_type_int["cluster"] ="overall"

df_aggregate_overall_type_int = df_aggregate_overall_type_int[["cluster", "rental_type", "price"]]

#combine datasets
full_type = pd.concat([df_aggregate_overall_type_int, df_aggregate_cluster_type_int])

full_type.dropna(inplace=True)

full_type.reset_index(inplace=True)

full_type.drop(columns="index", inplace=True)

full_type["Measure"] ="rental_type"

full_type.rename(columns ={"rental_type":"measure_value"}, inplace=True)

In [247]:
full_type

,cluster,measure_value,price,Measure
0,overall,apartment,1833.340715,rental_type
1,overall,condo,2216.335118,rental_type
2,overall,cottage,2149.800000,rental_type
3,overall,duplex,2561.084158,rental_type
4,overall,flat,2091.049020,rental_type
5,overall,house,2364.633295,rental_type
6,overall,land,2460.000000,rental_type
7,overall,loft,2636.027119,rental_type
8,overall,suite,2057.682645,rental_type
9,overall,townhouse,2876.559829,rental_type


## Image

not including in final dataset given the low n of one group

In [248]:
#find mean by image presence and cluster 
df_aggregate_cluster_image= combined_df.groupby(["cluster", "image_boolean"])["price"].mean()

df_aggregate_cluster_image_int = df_aggregate_cluster_image.reset_index()

#find mean by rental type /overall
df_aggregate_overall_image= combined_df.groupby("image_boolean")["price"].mean()

df_aggregate_overall_image_int= df_aggregate_overall_image.reset_index()

df_aggregate_overall_image_int["cluster"] ="overall"

df_aggregate_overall_image_int = df_aggregate_overall_image_int[["cluster", "image_boolean", "price"]]

#combine datasets
full_image = pd.concat([df_aggregate_overall_image_int, df_aggregate_cluster_image_int])

full_image.dropna(inplace=True)

full_image.reset_index(inplace=True)

full_image.drop(columns="index", inplace=True)

full_image["Measure"] ="image"

full_image.rename(columns ={"image_boolean":"measure_value"}, inplace=True)

In [249]:
full_image

,cluster,measure_value,price,Measure
0,overall,1,2152.210861,image
1,0.0,1,2084.523714,image
2,1.0,1,2288.649825,image
3,2.0,1,2090.229695,image
4,Not Feasible to predict,1,3167.838710,image


In [250]:
df["image"].count()

10949

In [251]:
df["FSA"].count()

11201

## Furnished 

In [252]:
#find mean by furnished presence and cluster 
df_aggregate_cluster_furnished= combined_df.groupby(["cluster", "furnished_update"])["price"].mean()

df_aggregate_cluster_furnished_int = df_aggregate_cluster_furnished.reset_index()

#find mean by furnished presence /overall
df_aggregate_overall_furnished= combined_df.groupby("furnished_update")["price"].mean()

df_aggregate_overall_furnished_int= df_aggregate_overall_furnished.reset_index()

df_aggregate_overall_furnished_int["cluster"] ="overall"

df_aggregate_overall_furnished_int = df_aggregate_overall_furnished_int[["cluster", "furnished_update", "price"]]

#combine datasets
full_furnished = pd.concat([df_aggregate_overall_furnished_int, df_aggregate_cluster_furnished_int])

full_furnished.dropna(inplace=True)

full_furnished.reset_index(inplace=True)

full_furnished.drop(columns="index", inplace=True)

full_furnished["Measure"] ="furnished"

full_furnished.rename(columns ={"furnished_update":"measure_value"}, inplace=True)

In [253]:
full_furnished

,cluster,measure_value,price,Measure
0,overall,NO,2183.418283,furnished
1,overall,NOT_MENTIONED,2136.482189,furnished
2,overall,YES,2191.036229,furnished
3,0.0,NO,2065.718310,furnished
4,0.0,NOT_MENTIONED,2112.583825,furnished
5,0.0,YES,2006.782101,furnished
6,1.0,NO,2318.634615,furnished
7,1.0,NOT_MENTIONED,2205.228078,furnished
8,1.0,YES,2481.222222,furnished
9,2.0,NO,2133.425325,furnished


In [268]:
combined_df.groupby("furnished_update").count()

,id,cluster,image_boolean,pet_friendly_cl,FSA,bathrooms,bedrooms,description,furnished,image,...,long,pet_friendly,post_published_date,postal_code,price,rental_type,source,sqft,title,url
furnished_update,,,,,,,,,,,,,,,,,,,,,
NO,1083,1083,1083,1083,1083,1083,1083,1083,1083,1080,...,1083,1083,1083,1083,1083,1083,1083,1083,1083,1083
NOT_MENTIONED,7804,7804,7804,7804,7804,7804,7804,7804,5179,7694,...,7804,7804,7804,7804,7804,7804,7804,7804,7804,7804
YES,2291,2291,2291,2291,2291,2291,2291,2291,2291,2152,...,2291,2291,2291,2291,2291,2291,2291,2291,2291,2291


## pets

In [254]:
#find mean by pets presence and cluster 
df_aggregate_cluster_pets= combined_df.groupby(["cluster", "pet_friendly_cl"])["price"].mean()

df_aggregate_cluster_pets_int = df_aggregate_cluster_pets.reset_index()

#find mean by pets type /overall
df_aggregate_overall_pets= combined_df.groupby("pet_friendly_cl")["price"].mean()

df_aggregate_overall_pets_int= df_aggregate_overall_pets.reset_index()

df_aggregate_overall_pets_int["cluster"] ="overall"

df_aggregate_overall_pets_int = df_aggregate_overall_pets_int[["cluster", "pet_friendly_cl", "price"]]

#combine datasets
full_pets = pd.concat([df_aggregate_overall_pets_int, df_aggregate_cluster_pets_int])

full_pets.dropna(inplace=True)

full_pets.reset_index(inplace=True)

full_pets.drop(columns="index", inplace=True)

full_pets["Measure"] ="pets"

full_pets.rename(columns ={"pet_friendly_cl":"measure_value"}, inplace=True)


In [255]:
full_pets

,cluster,measure_value,price,Measure
0,overall,False,2180.490030,pets
1,overall,True,2090.303281,pets
2,0.0,True,2084.523714,pets
3,1.0,False,2288.649825,pets
4,2.0,False,2090.229695,pets
5,Not Feasible to predict,False,2597.038462,pets
6,Not Feasible to predict,True,6136.000000,pets


## Bring it all together

In [256]:
final = pd.concat([full_bathrooms, full_bedrooms, full_type, full_furnished, full_pets])

In [257]:
len(final)

153

In [262]:
#move into json file

final.to_csv("ClusteAgg.csv")

In [263]:
DF =pd.read_csv("ClusteAgg.csv")

In [264]:
DF=DF.drop(columns="Unnamed: 0")

In [265]:
DF.to_json("ClusteAgg.json")

# Counts by drivers/clusters

In [269]:
#find mean by no. of bathrooms and cluster 
df_aggregate_cluster_bathrooms= combined_df.groupby(["cluster", "bathrooms"])["price"].count()

In [270]:
df_aggregate_cluster_bathrooms

cluster                  bathrooms
0.0                      0.0           125
                         1.0          2496
                         1.5           105
                         2.0           663
                         2.5            28
                         3.0            40
                         3.5             3
                         4.0             5
                         5.0             2
                                        33
1.0                      0.0             6
                         1.0          2182
                         1.5           110
                         2.0          1052
                         2.5             7
                         3.0            32
                                        35
2.0                      0.0            38
                         1.0          3164
                         1.5           113
                         2.0           503
                         2.5            27
                   

In [271]:
#find mean by no. of bathrooms and cluster 
df_aggregate_cluster_bedrooms= combined_df.groupby(["cluster", "bedrooms"])["price"].count()

In [ ]:
df_aggregate_cluster_bedrooms